## Random Forest
1. Bagging - обучение дерева на датасете из случайных строк изначального датасета. Строки могут пересекаться, а могут не пересекаться
2. Random subspaces - использует произвольные столбцы. Таким способом обучение происходит быстрее, а дерево становиться более устойчивым
3. Random patches - использует произвольные строки и произвольные столбцы

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.ensemble import BaggingClassifier, StackingClassifier, AdaBoostClassifier
from sklearn.datasets import make_classification
import sklearn.datasets as sd
import sklearn.model_selection as sm
import graphviz

In [3]:
test_dat_path = '../datasets/test_add_forest.dat'

f = open(test_dat_path, 'r')

XARR1 = []
XARR2 = []
XARR3 = []
XARR4 = []
XARR5 = []
y = []

for line in f:
    cols = line.split()
    if cols:

        XARR1.append([cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], cols[6], cols[7], 0, 0])
        XARR2.append([cols[0], cols[1], cols[2], cols[3], 0, 0, cols[6], cols[7], cols[8], cols[9]])
        XARR3.append([0, 0, cols[2], cols[3], cols[4], cols[5], cols[6], cols[7], cols[8], cols[9]])
        XARR4.append([0, cols[1], cols[2], 0, cols[4], cols[5], cols[6], cols[7], cols[8], cols[9]])
        XARR5.append([cols[0], cols[1], cols[2], cols[3], 0, cols[5], cols[6], cols[7], cols[8], cols[9]])
        y.append(cols[10])

dt1 = tree.DecisionTreeClassifier(max_depth=2)
dt1 = dt1.fit(XARR1, y)

dt2 = tree.DecisionTreeClassifier(max_depth=2)
dt2 = dt2.fit(XARR2, y)

dt3 = tree.DecisionTreeClassifier(max_depth=2)
dt3 = dt3.fit(XARR3, y)

dt4 = tree.DecisionTreeClassifier(max_depth=2)
dt4 = dt4.fit(XARR4, y)

dt5 = tree.DecisionTreeClassifier(max_depth=2)
dt5 = dt5.fit(XARR5, y)

In [4]:
good = 0
total = 0
i = 0

f = open(test_dat_path, 'r')

for line in f:
    cols = line.split()
    if cols:
        EX = [cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], cols[6], cols[7], cols[8], cols[9]]
        r1 = dt1.predict([EX])
        r2 = dt2.predict([EX])
        r3 = dt3.predict([EX])
        r4 = dt4.predict([EX])
        r5 = dt5.predict([EX])
        z = int((int(r1[0]) + int(r2[0]) + int(r3[0]) + int(r4[0]) + int(r5[0])) / 5)

        if z == int(cols[10]):
            good += 1
        total += 1

print(str(good/ total * 100), '%')
print(f'good: {good}')
print(f'total: {total}')

65.31963470319636 %
good: 2861
total: 4380


## BaggingClassifier from sklearn

In [5]:
test_dat_path = '../datasets/test_add_forest.dat'

f = open(test_dat_path, 'r')

X = []
y = []

for line in f:
    cols = line.split()
    if cols:
        X.append([cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], cols[6], cols[7], cols[8], cols[9]])
        y.append(cols[10])

bagging_clf = BaggingClassifier(tree.DecisionTreeClassifier(max_depth=2), n_estimators=10)
bagging_clf = bagging_clf.fit(X, y)

print(bagging_clf.score(X, y))

0.7924657534246575


## Stacking
**Stacking** - Для вычисления итогового результата обучается дополнительное решающее дерево, которое получает данные с деревьев первого этапа и выдает ответ

In [6]:
test_dat_path = '../datasets/test_add_forest.dat'

f = open(test_dat_path, 'r')

XARR1 = []
XARR2 = []

y = []

for line in f:
    cols = line.split()
    if cols:

        XARR1.append([cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], cols[6], cols[7], 0, 0])
        XARR2.append([cols[0], cols[1], cols[2], cols[3], 0, 0, cols[6], cols[7], cols[8], cols[9]])
        y.append(cols[10])

dt1 = tree.DecisionTreeClassifier(max_depth=3)
dt1 = dt1.fit(XARR1, y)

dt2 = tree.DecisionTreeClassifier(max_depth=3)
dt2 = dt2.fit(XARR2, y)

In [7]:
f = open(test_dat_path, 'r')

X2 = []
y2 = []

for line in f:
    cols = line.split()
    if cols:
        EX = [cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], cols[6], cols[7], cols[8], cols[9]]
        r1 = dt1.predict([EX])
        r2 = dt2.predict([EX])
        X2.append([r1[0], r2[0]])
        y2.append(cols[10])

clf3 = tree.DecisionTreeClassifier(max_depth=2)
clf3.fit(X2, y2)

DecisionTreeClassifier(max_depth=2)

In [8]:
f = open(test_dat_path, 'r')

good = 0
total = 0
for line in f:
    cols = line.split()
    if cols:
        EX = [cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], cols[6], cols[7], cols[8], cols[9]]
        r1 = dt1.predict([EX])
        r2 = dt2.predict([EX])
        EX3 = [r1[0], r2[0]]
        ans = clf3.predict([EX3])

        if int(ans[0]) == int(cols[10]):
            good += 1
        total += 1

In [9]:
print(str(good/ total * 100), '%')
print(f'good: {good}')
print(f'total: {total}')

79.20091324200914 %
good: 3469
total: 4380


## StackingClassifier from sklearn

In [10]:
test_dat_path = '../datasets/test_add_forest.dat'

f = open(test_dat_path, 'r')

XARR3 = []

y3 = []

for line in f:
    cols = line.split()
    if cols:

        XARR3.append([cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], cols[6], cols[7], cols[8], cols[9]])
        y3.append(cols[10])

stacking_clf = StackingClassifier(estimators=[('t1', tree.DecisionTreeClassifier(max_depth=5)),
                                              ('t2', tree.DecisionTreeClassifier(max_depth=5)),
                                              ('t3', tree.DecisionTreeClassifier(max_depth=5))],
                                  final_estimator=tree.DecisionTreeClassifier(max_depth=2))

stacking_clf = stacking_clf.fit(XARR3, y3)
print(stacking_clf.score(XARR3, y3))

0.9164383561643835


## Boosting
**Boosting** - обучаем новое дерево, которое получает на вход исходный датасет и ответ с деревьев первого слоя

# Добавить код для бустинга

In [11]:
test_dat_path = '../datasets/test_add_forest.dat'

f = open(test_dat_path, 'r')

XARR1 = []
XARR2 = []

y = []

for line in f:
    cols = line.split()
    if cols:

        XARR1.append([cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], cols[6], cols[7], 0, 0])
        XARR2.append([cols[0], cols[1], cols[2], cols[3], 0, 0, cols[6], cols[7], cols[8], cols[9]])
        y.append(cols[10])

dt1 = tree.DecisionTreeClassifier(max_depth=3)
dt1 = dt1.fit(XARR1, y)

dt2 = tree.DecisionTreeClassifier(max_depth=3)
dt2 = dt2.fit(XARR2, y)

In [12]:
f = open(test_dat_path, 'r')

X2 = []
y2 = []

bX = []
bY = []
for line in f:
    cols = line.split()
    if cols:
        EX = [cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], cols[6], cols[7], cols[8], cols[9]]
        r1 = dt1.predict([EX])
        r2 = dt2.predict([EX])
        X2.append([r1[0], r2[0]])
        y2.append(cols[10])

clf3 = tree.DecisionTreeClassifier(max_depth=2)
clf3.fit(X2, y2)

DecisionTreeClassifier(max_depth=2)

## AdaBoost
Обучаем одно дерево на всем датасете, потом циклические обучаем новое дерево на основе полученные данных с предыдущего дерева и исходного датасета

In [13]:
X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0, shuffle=False)

clf = AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=1), n_estimators=100)

clf.fit(X, y)
clf.score(X, y)

0.96

### Using test_add dataset
# ToDo

In [16]:
f = open(test_dat_path, 'r')

X = []
y = []

for line in f:
    cols = line.split()
    if cols:
        X.append([cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], cols[6], cols[7], cols[8], cols[9]])
        y.append(cols[10])

In [ ]:
clf = AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=1), n_estimators=100)

clf.fit(X, y)
clf.score(X, y)

## Gradient Boosting

**жесткая классификация:** ответ решающего дерева - жесткий класс
**мягкая классфикация:** ответ решающего дерева - массив с вероятностью класса
**Регрессия:** дает грубый ответ, но действительный числа.

Градиентный бустинг позволяет решать задачи регрессии

In [68]:
dataset = sd.load_boston()
print(dataset.data.shape)
print(dataset.target.shape)

(506, 13)
(506,)
